# Run DRN

In [1]:
#from typing import Any
#from pytorch_lightning.utilities.types import STEP_OUTPUT
#from exploration.gnn_run import in_channels
# load data first
%cd /home/ltchen/gnnpp
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import argparse
import json
import pytorch_lightning as L
import torch
import wandb

from models.drn import DRN
from models.model_utils import EmbedStations
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
#from sklearn.preprocessing import StandardScaler
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch import nn

from utils.data import load_dataframes, summary_statistics
from utils.drn_utils import *
from models.loss import NormalCRPS

/home/ltchen/gnnpp


### Load dataframes for train, valid, test

In [2]:
dataframes = load_dataframes(mode="train", leadtime="24h") # train mode => for training nn? Wie wird das im Paper beschrieben?
dataframes = summary_statistics(dataframes) # wie sehen die daten von summary statistics aus? => wenn das nur die Daten von einer Station sind, dann über Zeitpunkte
dataframes.pop("stations") # .pop("stations") => entfernt den df mit stations, wofuer brauche ich die dann überhaupt? Grafik?

# test
for X, y in dataframes.values(): # wofuer?
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

train, valid_test = normalize_features(
    training_data=dataframes["train"], valid_test_data=[dataframes["test_rf"], dataframes["test_f"]]
)

train = drop_nans(train)
(test_rf, test_f) = valid_test
test_rf = drop_nans(test_rf)
test_f = drop_nans(test_f)

DIRECTORY = os.getcwd()
SAVEPATH = os.path.join(DIRECTORY, "explored_models/drn_24h/models")
#print(train[1].isna().sum()) #drop_nans does not work without summary_statistics

[INFO] Dataframes exist. Will load pandas dataframes.
[INFO] Calculating summary statistics for train
[INFO] Calculating summary statistics for test_rf
[INFO] Calculating summary statistics for test_f
[INFO] Normalizing features...


In [38]:
nans = train[1]["t2m"].isna().reset_index(drop=True)
print(train[0].index)
print(train[1].index)
#nans.index
#train[0][~nans]
#res = (train[0][~nans], train[1][~nans])
#res

RangeIndex(start=0, stop=4627216, step=1)
RangeIndex(start=0, stop=420656, step=1)


In [3]:
# one station only
# train
for i in train:
    print(i.shape)
    print(i.columns)

# valid_test
for (i, j) in valid_test:
    print(i.shape)
    print(i.columns)

(398866, 65)
Index(['station_id', 'model_orography', 'station_altitude', 'station_latitude',
       'station_longitude', 'cape_mean', 'cape_std', 'sd_mean', 'sd_std',
       'stl1_mean', 'stl1_std', 'swvl1_mean', 'swvl1_std', 't2m_mean',
       't2m_std', 'tcc_mean', 'tcc_std', 'tcw_mean', 'tcw_std', 'tcwv_mean',
       'tcwv_std', 'u10_mean', 'u10_std', 'u100_mean', 'u100_std', 'v10_mean',
       'v10_std', 'v100_mean', 'v100_std', 'vis_mean', 'vis_std', 'cp6_mean',
       'cp6_std', 'mn2t6_mean', 'mn2t6_std', 'mx2t6_mean', 'mx2t6_std',
       'p10fg6_mean', 'p10fg6_std', 'slhf6_mean', 'slhf6_std', 'sshf6_mean',
       'sshf6_std', 'ssr6_mean', 'ssr6_std', 'ssrd6_mean', 'ssrd6_std',
       'str6_mean', 'str6_std', 'strd6_mean', 'strd6_std', 'tp6_mean',
       'tp6_std', 'z_mean', 'z_std', 'q_mean', 'q_std', 'u_mean', 'u_std',
       'v_mean', 'v_std', 't_mean', 't_std', 'cos_doy', 'sin_doy'],
      dtype='object')
(398866, 3)
Index(['time', 'station_id', 't2m'], dtype='object')
(89304

## One Station

In [25]:
# train
one_station_X = train[0][train[0]["station_id"]==1]
one_station_y = train[1][train[1]["station_id"]==1]

one_station_X = one_station_X.drop("station_id", axis=1)
one_station_y = one_station_y.drop("station_id", axis=1)

print(one_station_X)
print(one_station_X.shape)
print(one_station_y.shape)

# test_rf
s1_test_rf_X = test_rf[0][test_rf[0]["station_id"]==1]
s1_test_rf_y = test_rf[1][test_rf[1]["station_id"]==1]

s1_test_rf_X = s1_test_rf_X.drop("station_id", axis=1)
s1_test_rf_y = s1_test_rf_y.drop("station_id", axis=1)

# test_f
s1_test_f_X = test_f[0][test_f[0]["station_id"]==1]
s1_test_f_y = test_f[1][test_f[1]["station_id"]==1]

s1_test_f_X = s1_test_f_X.drop("station_id", axis=1)
s1_test_f_y = s1_test_f_y.drop("station_id", axis=1)

        model_orography  station_altitude  station_latitude  \
1             -0.737002           -0.7786          1.016052   
123           -0.737002           -0.7786          1.016052   
245           -0.737002           -0.7786          1.016052   
367           -0.737002           -0.7786          1.016052   
489           -0.737002           -0.7786          1.016052   
...                 ...               ...               ...   
420047        -0.737002           -0.7786          1.016052   
420169        -0.737002           -0.7786          1.016052   
420291        -0.737002           -0.7786          1.016052   
420413        -0.737002           -0.7786          1.016052   
420535        -0.737002           -0.7786          1.016052   

        station_longitude  cape_mean  cape_std   sd_mean    sd_std  stl1_mean  \
1                -0.89124  -0.164696 -0.233030 -0.138945 -0.130389  -0.465046   
123              -0.89124  -0.127184 -0.138486 -0.138945 -0.130389  -0.443237   


### One Station MSE and CRPS NNs
Station (station_id=1) with one hidden layer and loss functions MSE or CRPS

In [30]:
class MSEStationNN(L.LightningModule):
    def __init__(self, in_feat, hidden_size, optimizer_class, optimizer_params):
        super(MSEStationNN, self).__init__()
        self.linear = torch.nn.Linear(in_features=in_feat, out_features=hidden_size)
        self.relu = torch.nn.ReLU()
        self.linear_t2m = torch.nn.Linear(in_features=hidden_size, out_features=1)

        self.loss = torch.nn.MSELoss()
        self.optimizer_class = optimizer_class
        self.optimizer_params = optimizer_params

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear_t2m(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y.flatten())
        self.log("train_loss", loss.item(), on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer_class(self.parameters(), **self.optimizer_params)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y.flatten())
        return loss

    def test_step(self, batch, batch_idx, dataloader_idx=0): # unterschied zwischen predict und test_step?
        x, y = batch # wieso hat test_step auch y? => um score zu berechnen
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y.flatten())
        return loss


In [31]:
class CRPSStationNN(L.LightningModule):
    def __init__(self, in_feat, hidden_size, optimizer_class, optimizer_params):
        super(CRPSStationNN, self).__init__()
        self.linear = torch.nn.Linear(in_features=in_feat, out_features=hidden_size)
        self.relu = torch.nn.ReLU()
        #self.linear_t2m = torch.nn.Linear(in_features=hidden_size, out_features=2) => wieso nicht direkt 2 outputs?
        self.softplus = torch.nn.Softplus()
        self.last_linear_mu = nn.Linear(in_features=hidden_size, out_features=1)
        self.last_linear_sigma = nn.Linear(in_features=hidden_size, out_features=1)

        self.loss_fn = NormalCRPS()
        self.optimizer_class = optimizer_class
        self.optimizer_params = optimizer_params

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear(x)
        x = self.relu(x)
        mu = self.last_linear_mu(x)
        sigma = self.softplus(self.last_linear_sigma(x))
        res = torch.cat([mu, sigma], dim=1)
        return res

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn.crps(mu_sigma=y_hat, y=y.flatten())
        self.log("train_loss", loss.item(), on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer_class(self.parameters(), **self.optimizer_params)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn.crps(mu_sigma=y_hat, y=y.flatten())
        return loss

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn.crps(mu_sigma=y_hat, y=y.flatten())
        print(f'test_loss: {loss}')
        return {'loss': loss}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, _ = batch
        y_hat = self.forward(x)
        return y_hat


### Train One Station NN (MSE or CRPS)

In [32]:
with wandb.init(
    project="exploration",
    id = f"training_run_24h_crps",
    tags=["exploration"],
):

    y_scaler = StandardScaler(with_std=False) # wieso scalen wir überhaupt? => robuster?
    y_scaler = y_scaler.fit(one_station_y[["t2m"]])

    batch_size = 512
    hidden_size=128
    lr=0.0002
    max_epochs=31
    in_feat = one_station_X.shape[1]

    one_station_train_ds = TensorDataset(torch.Tensor(one_station_X.to_numpy()), torch.Tensor(y_scaler.transform(one_station_y[["t2m"]])))
    one_station_loader = DataLoader(one_station_train_ds, batch_size=batch_size, shuffle=True)

    s1_test_rf_ds = TensorDataset(torch.Tensor(s1_test_rf_X.to_numpy()), torch.Tensor(y_scaler.transform(s1_test_rf_y[["t2m"]])))
    s1_test_rf_loader = DataLoader(s1_test_rf_ds, batch_size=batch_size, shuffle=False)

    one_station_nn = CRPSStationNN(
        in_feat=in_feat,
        hidden_size=hidden_size,
        optimizer_class=AdamW,
        optimizer_params={"lr": lr}
    )

    wandb_logger = WandbLogger(project="one_station_crps")

    os_checkpoint_callback = ModelCheckpoint(
        dirpath=SAVEPATH, filename=f"run_24h", monitor="train_loss", mode="min", save_top_k=1
    )

    one_station_trainer = L.Trainer(
        max_epochs=max_epochs,
        log_every_n_steps=1,
        accelerator="gpu",
        enable_model_summary=True,
        logger=wandb_logger,
        callbacks=os_checkpoint_callback,
    )

    value = one_station_trainer.fit(model=one_station_nn, train_dataloaders=one_station_loader)

    final_loss = one_station_trainer.logged_metrics["train_loss_step"] # nochmal step und epoch nachschauen
    print("Final MSE Loss:", final_loss)


# wo finde ich den tatsaechlichen wert? => bei test, jetzt wird nur das Modell trainiert

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: C

Epoch 30: 100%|██████████| 7/7 [00:00<00:00, 57.77it/s, v_num=crps, train_loss_step=1.320, train_loss_epoch=1.280]

`Trainer.fit` stopped: `max_epochs=31` reached.


Epoch 30: 100%|██████████| 7/7 [00:00<00:00, 52.44it/s, v_num=crps, train_loss_step=1.320, train_loss_epoch=1.280]
Final MSE Loss: tensor(1.3195)


epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
train_loss_epoch,███▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train_loss_step,█▇██▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇███
epoch,30
train_loss_epoch,1.28279
train_loss_step,1.31945
trainer/global_step,216


### Validate and test one station NN (MSE and CRPS)

In [33]:
# validation and test for both mse and crps
s1_test_f_ds = TensorDataset(torch.Tensor(s1_test_f_X.to_numpy()), torch.Tensor(y_scaler.transform(s1_test_f_y[["t2m"]])))
s1_test_f_loader = DataLoader(s1_test_f_ds, batch_size=batch_size, shuffle=False)

loss = one_station_trainer.test(model=one_station_nn, dataloaders=s1_test_f_loader)
print(loss)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 225.69it/s]
[{}]


In [34]:
preds_list = []
preds = one_station_trainer.predict(model=one_station_nn, dataloaders=s1_test_f_loader)
preds = torch.cat(preds, dim=0)
print(f"preds.shape: {preds.shape}")
print(f"first preds: {preds[0]}")
# Reverse transform of the y_scaler (only on the mean)
preds[:, 0] = torch.Tensor(y_scaler.inverse_transform(preds[:, 0].view(-1, 1))).flatten()

preds_list.append(preds)
print(f"preds_list length: {preds_list[0][0]}")
targets = s1_test_f_y
targets = torch.Tensor(targets.t2m.values)

stacked = torch.stack(preds_list)
print(f"stacked shape: {stacked.shape}")
#final_preds = torch.mean(stacked, dim=0)

final_preds = stacked[0]

res = one_station_nn.loss_fn.crps(final_preds, targets)
print(f"final pred: {final_preds[0]}, targets: {targets[0]}")
print(res)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 218.93it/s]
preds.shape: torch.Size([730, 2])
first preds: tensor([-3.1140,  1.9057])
preds_list length: tensor([279.2640,   1.9057])
stacked shape: torch.Size([1, 730, 2])
final pred: tensor([279.2640,   1.9057]), targets: 275.25
tensor(1.2613)


## All stations
Deterministic NN with one hidden layer using MSE as loss and embeddings

In [8]:
# nn mse loss with lightning
class MyDRN(L.LightningModule):
    def __init__(self, hidden_size, embedding_dim, in_feat, optimizer_class, optimizer_params):
        super(MyDRN, self).__init__()
        self.embedding = EmbedStations(num_stations_max=122, embedding_dim=embedding_dim)
        self.linear = torch.nn.Linear(in_features=in_feat, out_features=hidden_size)
        self.relu = torch.nn.ReLU()
        self.linear_t2m = torch.nn.Linear(in_features=hidden_size, out_features=1) # output t2m value

        self.loss = torch.nn.MSELoss()
        self.optimizer_class = optimizer_class
        self.optimizer_params = optimizer_params

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #print(x.shape) # (8, 65)
        x = self.embedding(x)
        #print(f"After embedding: {x.shape}") # (8, 84)
        x = self.linear(x)
        #print(x.shape) # (8, 64)
        x = self.relu(x)
        #print(x.shape)
        x = self.linear_t2m(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y.flatten()) # why y.flatten()?
        self.log("train_loss", loss.item(), on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer_class(self.parameters(), **self.optimizer_params)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y.flatten())
        self.log("validation_loss", loss.item(), on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx, dataloader_idx=0): # unterschied zwischen predict und test_step?
        x, y = batch # wieso hat test_step auch y?
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y.flatten())
        self.log("test_loss", loss.item(), on_epoch=True, prog_bar=True)
        return loss

In [3]:
# nn crps loss with lightning
class CRPSDRN(L.LightningModule):
    def __init__(self, embedding_dim, in_channels, hidden_channels, optimizer_class, optimizer_params):
        super(CRPSDRN, self).__init__()
        self.hidden_channels = hidden_channels
        self.num_layers = len(hidden_channels)

        self.embedding = EmbedStations(num_stations_max=122, embedding_dim=embedding_dim)
        self.linear = nn.ModuleList()
        for hidden_size in self.hidden_channels:
            self.linear.append(nn.Linear(in_features=in_channels, out_features=hidden_size))
            in_channels = hidden_size
        self.relu = torch.nn.ReLU()
        self.softplus = torch.nn.Softplus()
        self.last_linear_mu = nn.Linear(in_features=hidden_size, out_features=1)
        self.last_linear_sigma = nn.Linear(in_features=hidden_size, out_features=1)

        self.loss_fn = NormalCRPS()
        self.optimizer_class = optimizer_class
        self.optimizer_params = optimizer_params

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x) #(8, 65)
        for layer in self.linear:
            x = layer(x)
            print(x.shape)
            x = self.relu(x)
            print(x.shape)
        mu = self.last_linear_mu(x)
        x = self.relu(x) #(8, 64)
        mu = self.last_linear_mu(x)
        sigma = self.softplus(self.last_linear_sigma(x))
        res = torch.cat([mu, sigma], dim=1)
        return res

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn.crps(mu_sigma=y_hat, y=y.flatten())
        self.log("train_loss", loss.item(), on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer_class(self.parameters(), **self.optimizer_params)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn.crps(mu_sigma=y_hat, y=y.flatten())
        self.log("validation_loss", loss.item(), on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx, dataloader_idx=0): # unterschied zwischen predict und test_step?
        x, y = batch # wieso hat test_step auch y?
        y_hat = self.forward(x)
        loss = self.loss_fn.crps(mu_sigma=y_hat, y=y.flatten())
        self.log("test_loss", loss.item(), on_epoch=True, prog_bar=True)
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, _ = batch
        y_hat = self.forward(x)
        return y_hat


In [4]:
# import json file to config
from dataclasses import dataclass

CHECKPOINT_FOLDER = os.path.join(DIRECTORY, "trained_models/drn_24h")
JSONPATH = os.path.join(CHECKPOINT_FOLDER, "params.json")

with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
config = args_dict
# check if it works, if not: see in drn_eval
'''
@dataclass
class DummyConfig:
    pass

for key, value in args_dict.items():
    setattr(DummyConfig, key, value)

config = DummyConfig()
print("[INFO] Starting eval with config: ", args_dict)
'''
'''params.json
{"batch_size":8,
"gnn_hidden":265,
"gnn_layers":2,
"heads":8,
"lr":0.0002,
"max_dist":100,
"max_epochs": 31}
'''


[INFO] Loading /home/ltchen/gnnpp/trained_models/drn_24h/params.json


'params.json\n{"batch_size":8,\n"gnn_hidden":265,\n"gnn_layers":2,\n"heads":8,\n"lr":0.0002,\n"max_dist":100,\n"max_epochs": 31}\n'

### Train All_station NN

In [5]:
with wandb.init(
    project="reproduction",
    id = f"training_run_24h_crps",
    config=config,
    tags=["exploration"],
):
    config=wandb.config
    y_scaler = StandardScaler(with_std=False) # wieso scalen wir überhaupt? => robuster?
    y_scaler = y_scaler.fit(train[1][["t2m"]])

    #batch_size =2048
    #hidden_size=128
    #lr=0.0002
    #max_epochs=31

    embed_dim = 20
    in_channels = train[0].shape[1] + embed_dim - 1


    train_dataset = TensorDataset(torch.Tensor(train[0].to_numpy()), torch.Tensor(y_scaler.transform(train[1][["t2m"]])))
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

    mydrn = CRPSDRN(
        embedding_dim=embed_dim,
        in_channels=in_channels,
        hidden_channels=config.hidden_channels,

        optimizer_class=AdamW,
        optimizer_params=dict(lr=config.lr),
)

    wandb_logger = WandbLogger(project="all_station_crps")

    checkpoint_callback = ModelCheckpoint(
        dirpath=SAVEPATH, filename=f"run_24h", monitor="train_loss", mode="min", save_top_k=1
    )

    trainer = L.Trainer(
        max_epochs=config.max_epochs,
        log_every_n_steps=10,
        accelerator="gpu",
        enable_progress_bar=True,
        enable_model_summary=True,
        logger=wandb_logger,
        callbacks=checkpoint_callback,
)

    trainer.fit(model=mydrn, train_dataloaders=train_loader)

    final_loss = trainer.logged_metrics["train_loss_step"]
    print("Final MSE Loss:", final_loss)

wandb: Currently logged in as: leachen (leachen_thesis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/s

Epoch 0:   0%|          | 0/98 [00:00<?, ?it/s] torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   1%|          | 1/98 [00:00<01:07,  1.45it/s, v_num=crps, train_loss_step=4.800]torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   2%|▏         | 2/98 [00:00<00:34,  2.75it/s, v_num=crps, train_loss_step=3.890]torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   3%|▎         | 3/98 [00:00<00:24,  3.94it/s, v_num=crps, train_loss_step=2.850]torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   4%|▍         | 4/98 [00:00<00:19,  4.91it/s, v_num=crps, train_loss_step=2.010]torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   5%|▌         | 5/98 [00:00<00:15,  5.90it/s, v_num=crps, train_loss_step=1.720]torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   6%|▌         | 6/98 [00:01<00:16,  5.53it/s, v_num=crps, train_loss_step=1.880]torch.Size([4096, 256])
torch.Size([4096, 256])
Epoch 0:   7%|▋         | 7/98 [00:01<00:14,  6.11it/s, v_num=crps, train_lo

`Trainer.fit` stopped: `max_epochs=26` reached.


Epoch 25: 100%|██████████| 98/98 [00:05<00:00, 17.33it/s, v_num=crps, train_loss_step=0.624, train_loss_epoch=0.596]
Final MSE Loss: tensor(0.6243)


epoch,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss_epoch,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▄▃▃▃▂▃▂▃▄▂▂▂▂▂▂▂▂▂▁▁▁▃▂▃
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇███
epoch,25
train_loss_epoch,0.59555
train_loss_step,0.59957
trainer/global_step,2547


In [7]:
# eval (aber wo ist validate?)
test_rf_dataset = TensorDataset(torch.Tensor(test_rf[0].to_numpy()), torch.Tensor(y_scaler.transform(test_rf[1][["t2m"]])))
test_rf_loader = DataLoader(test_rf_dataset, batch_size=config.batch_size, shuffle=False)

test_f_dataset = TensorDataset(torch.Tensor(test_f[0].to_numpy()), torch.Tensor(y_scaler.transform(test_f[1][["t2m"]])))
test_f_loader = DataLoader(test_f_dataset, batch_size=config.batch_size, shuffle=False)

trainer = L.Trainer(
    log_every_n_steps=1, accelerator="gpu", enable_progress_bar=True, enable_model_summary=False
)
preds_list = []

preds = trainer.predict(model=mydrn, dataloaders=test_rf_loader)
preds = torch.cat(preds, dim=0)
# Reverse transform of the y_scaler (only on the mean)
preds[:, 0] = torch.Tensor(y_scaler.inverse_transform(preds[:, 0].view(-1, 1))).flatten()
preds_list.append(preds)

targets = test_rf[1]
targets = torch.Tensor(targets.t2m.values)

stacked = torch.stack(preds_list)
final_preds = torch.mean(stacked, dim=0)

res = mydrn.loss_fn.crps(final_preds, targets)
print("#############################################")
print("#############################################")
print(f"final crps: {res.item()}")
print("#############################################")
print("#############################################")

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 0/19 [00:00<?, ?it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:   5%|▌         | 1/19 [00:00<00:00, 371.51it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  11%|█         | 2/19 [00:00<00:00, 93.72it/s] torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  16%|█▌        | 3/19 [00:00<00:00, 75.02it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  21%|██        | 4/19 [00:00<00:00, 68.57it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  26%|██▋       | 5/19 [00:00<00:00, 64.49it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  32%|███▏      | 6/19 [00:00<00:00, 62.66it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  37%|███▋      | 7/19 [00:00<00:00, 60.81it/s]torch.Size([4096, 256])
torch.Size([4096, 256])
Predicting DataLoader 0:  42%|████▏     | 8/19 [00:00<

## DRN for 72h leadtime

In [ ]:
# dataloader and preprocessing
# NN object, train, test

## DRN for 120h leadtime

In [ ]:
# dataloader and preprocessing
# NN object, train, test

## MyDRN without wandb and without saving

In [19]:
# MyDRN train without wandb without saving

y_scaler = StandardScaler(with_std=False)
y_scaler = y_scaler.fit(train[1][["t2m"]])


train_dataset = TensorDataset(
    torch.Tensor(train[0].to_numpy()), torch.Tensor(y_scaler.transform(train[1][["t2m"]]))
)

#from params.json best_24h
batch_size = 2048
hidden_size=128
lr=0.0002
max_epochs=31

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

embed_dim = 20
in_feat = train[0].shape[1] + embed_dim - 1

mydrn = MyDRN(
    hidden_size=hidden_size,
    embedding_dim=embed_dim,
    in_feat=in_feat,
    optimizer_class=AdamW,
    optimizer_params=dict(lr=lr),
)

checkpoint_callback = ModelCheckpoint(
    # dirpath=SAVEPATH, filename=f"run_{args.id}", monitor="train_loss", mode="min", save_top_k=1
    dirpath=SAVEPATH, filename=f"run_24h", monitor="train_loss", mode="min", save_top_k=1
)

trainer = L.Trainer(
    max_epochs=max_epochs,
    log_every_n_steps=50,
    accelerator="gpu",
    enable_progress_bar=True,
    enable_model_summary=True,
    callbacks=checkpoint_callback,
)

trainer.fit(model=mydrn, train_dataloaders=train_loader)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-package

Epoch 0:   3%|▎         | 5/195 [00:00<00:05, 32.69it/s, v_num=28, train_loss_step=42.50]

/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([2048])) that is different to the input size (torch.Size([2048, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1:   3%|▎         | 5/195 [00:00<00:05, 34.61it/s, v_num=28, train_loss_step=41.90, train_loss_epoch=41.20]  

/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([1554])) that is different to the input size (torch.Size([1554, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 30: 100%|██████████| 195/195 [00:05<00:00, 37.23it/s, v_num=28, train_loss_step=39.70, train_loss_epoch=41.20]

`Trainer.fit` stopped: `max_epochs=31` reached.


Epoch 30: 100%|██████████| 195/195 [00:05<00:00, 37.21it/s, v_num=28, train_loss_step=39.70, train_loss_epoch=41.20]


## DRN with given DRN

In [37]:
# with given models - funktioniert fuer summary statistics, aber nicht ohne? => woran liegt das?
DIRECTORY = os.getcwd()
JSONPATH = os.path.join(DIRECTORY, "trained_models/drn_24h/params.json")
SAVEPATH = os.path.join(DIRECTORY, "trained_models/drn_24h/models")

with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)

with wandb.init(
    project="multigraph",
    # id=f"training_run_drn_{args_dict['leadtime']}_{args.id}",
    id = f"training_run_{args_dict['leadtime']}",
    config=args_dict,
    tags=["final_training"],
):
    config=wandb.config
    dataframes = load_dataframes(mode="train", leadtime=config.leadtime)
    dataframes = summary_statistics(dataframes)
    dataframes.pop("stations")

    # print(list(dataframes.values()))
    for df in dataframes.values():
        print(type(df))

    for X, y in dataframes.values():
        X.reset_index(drop=True, inplace=True)
        y.reset_index(drop=True, inplace=True)

    train, valid_test = normalize_features(
        training_data=dataframes["train"], valid_test_data=[dataframes["test_rf"], dataframes["test_f"]]
    )

    print(f"dataframes['train']: {dataframes['train']}")
    print(f"train: {train}")

    train = drop_nans(train)

    y_scaler = StandardScaler(with_std=False)
    y_scaler = y_scaler.fit(train[1][["t2m"]])

    train_dataset = TensorDataset(
        torch.Tensor(train[0].to_numpy()), torch.Tensor(y_scaler.transform(train[1][["t2m"]]))
    )

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

    embed_dim = 20 # why 20? => embed stations - instead of station_id - map into a latent vector space
    in_channels = train[0].shape[1] + embed_dim - 1

    drn = DRN(
        in_channels=in_channels,
        hidden_channels=config.hidden_channels,
        embedding_dim=embed_dim,
        optimizer_class=AdamW,
        optimizer_params=dict(lr=config.lr),
    )
    wandb_logger = WandbLogger(project="multigraph")
    checkpoint_callback = ModelCheckpoint(
        # dirpath=SAVEPATH, filename=f"run_{args.id}", monitor="train_loss", mode="min", save_top_k=1
        dirpath=SAVEPATH, filename=f"run_24h", monitor="train_loss", mode="min", save_top_k=1
    )
    trainer = L.Trainer(
        max_epochs=config.max_epochs,
        log_every_n_steps=1,
        accelerator="gpu",
        enable_progress_bar=True,
        enable_model_summary=True,
        logger=wandb_logger,
        callbacks=checkpoint_callback,
    )
    trainer.fit(model=drn, train_dataloaders=train_loader)


[INFO] Loading /home/ltchen/gnnpp/trained_models/drn_24h/params.json


[INFO] Dataframes exist. Will load pandas dataframes.
[INFO] Calculating summary statistics for train
[INFO] Calculating summary statistics for test_rf
[INFO] Calculating summary statistics for test_f
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
[INFO] Normalizing features...


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/ltchen/.conda/envs/gnn_env/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: C

dataframes['train']: (        station_id  model_orography  station_altitude  station_latitude  \
0                0        -0.738289         -0.764101          1.382904   
1                1        -0.737002         -0.778600          1.016052   
2                2        -0.731851         -0.733171          1.571141   
3                3        -0.728793         -0.765712          1.661951   
4                4        -0.724769         -0.761846          0.884948   
...            ...              ...               ...               ...   
420651         117         0.914135          0.298485         -2.911765   
420652         118         1.443053          0.598123         -1.881973   
420653         119         2.338639          0.646451         -2.021799   
420654         120         4.799571          3.994016         -2.028314   
420655         121         5.913903          4.345205         -2.201382   

        station_longitude  cape_mean  cape_std   sd_mean    sd_std  stl1_mean

`Trainer.fit` stopped: `max_epochs=26` reached.


Epoch 25: 100%|██████████| 98/98 [00:05<00:00, 17.92it/s, v_num=_24h, train_loss_step=0.580, train_loss_epoch=0.597]


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▆▆▇▇▇▇▇▇▇████
train_loss_epoch,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▆▄▄▄▃▃▃▂▃▃▂▃▂▂▂▂▃▂▂▂▂▂▂▃▂▂▂▂▂▁▁▂▁▂▂▁▂▁▂
trainer/global_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
epoch,25
train_loss_epoch,0.59687
train_loss_step,0.58027
trainer/global_step,2547


In [51]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
